In [1]:
import numpy as np

In [3]:
X = np.load("../data/X.npy")

In [8]:
def get_day_from_sincos(day_sin, day_cos, is_leap_year=False):
  """
  Calculates the approximate day of the year from its sine and cosine encoding.
  Corrected version to handle day 1 properly.

  Args:
    day_sin: The sine component of the encoded day.
    day_cos: The cosine component of the encoded day.
    is_leap_year: Boolean indicating if the year should be treated as a leap year (366 days).
                  Defaults to False (365 days).

  Returns:
    An integer representing the approximate day of the year (1-365 or 1-366).
  """
  # Calculate the angle using arctan2(y, x) -> arctan2(sin, cos)
  angle = np.arctan2(day_sin, day_cos)

  # Convert the angle range from [-pi, pi] to [0, 2*pi]
  # Handle potential array input
  if isinstance(angle, (np.ndarray)):
      angle[angle < 0] += 2 * np.pi
  elif angle < 0:
      angle += 2 * np.pi

  # Determine the number of days in the year
  days_in_year = 366.0 if is_leap_year else 365.0

  # Reverse the normalization used in the forward calculation:
  # angle = (day_of_year / days_in_year) * 2 * pi
  # day_of_year = (angle / (2 * pi)) * days_in_year
  year_fraction = angle / (2 * np.pi)
  approx_day_float = year_fraction * days_in_year

  # Use ceiling function to map correctly. Add a small epsilon
  # before ceiling for values very close to an integer boundary,
  # though ceil should handle 1.0 -> 1, 1.0001 -> 2 etc.
  # Let's test ceil directly first.
  approx_day_int = np.ceil(approx_day_float).astype(int)

  # Handle the edge case where angle is exactly 0 (or numerically close)
  # which corresponds to the very start, mapping should be day 1.
  # If approx_day_int is 0 after ceiling (e.g., if approx_day_float was ~0), set to 1.
  if isinstance(approx_day_int, (np.ndarray)):
      approx_day_int[approx_day_int == 0] = 1
  elif approx_day_int == 0:
      approx_day_int = 1

  # Clip to ensure result is within [1, days_in_year]
  approx_day_int = np.clip(approx_day_int, 1, int(days_in_year))


  return approx_day_int


In [9]:
first_day_sequence = X[0] # Shape (5, 24)
# Get sin/cos values (they are constant, so take from first hour)
day_sin_first = first_day_sequence[3, 0]
day_cos_first = first_day_sequence[4, 0]

# Calculate the day. Assuming the first day corresponds to 2023 (not a leap year).
# Our data processing started from 2023-01-01.
is_leap_first = False # 2023 is not a leap year
calculated_day_first = get_day_from_sincos(day_sin_first, day_cos_first, is_leap_year=is_leap_first)

print(f"\n--- First Day Sequence (Index 0) ---")
print(f"  Day Sin value: {day_sin_first:.6f}")
print(f"  Day Cos value: {day_cos_first:.6f}")
print(f"  Calculated Day of Year (assuming {'leap' if is_leap_first else 'non-leap'}): {calculated_day_first}")



--- First Day Sequence (Index 0) ---
  Day Sin value: 0.017213
  Day Cos value: 0.999852
  Calculated Day of Year (assuming non-leap): 1


: 